In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from ultralytics import YOLO
import cv2
import yaml
import os
import time

# Load your trained model - update path to your best weights
model_path = "/content/drive/MyDrive/Pengolahan Citra Digital/test_detect/best (1).pt"
model = YOLO(model_path)

# Load class names
with open("/content/drive/MyDrive/Pengolahan Citra Digital/combined_gunsnknifes/data.yaml", 'r') as f:
    data = yaml.safe_load(f)
    class_names = data['names']
    print("Classes:", class_names)

# Input and output video paths
input_video_path = "/content/drive/MyDrive/Pengolahan Citra Digital/test_detect/knife_testing4.mp4"  # replace with your video path
output_video_path = "/content/drive/MyDrive/Pengolahan Citra Digital/test_detect/knife_testing4_detect.mp4"

# Open the video
cap = cv2.VideoCapture(input_video_path)
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Initialize video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Process video frame by frame
frame_count = 0
start_time = time.time()

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # Increment frame counter
    frame_count += 1

    if frame_count % 10 == 0:  # Update progress every 10 frames
        elapsed_time = time.time() - start_time
        frames_left = total_frames - frame_count
        time_left = frames_left * elapsed_time / frame_count if frame_count > 0 else 0
        print(f"Processing frame {frame_count}/{total_frames} - ETA: {time_left:.1f}s")

    # Run detection on the frame
    results = model(frame, conf=0.30, iou=0.45)

    # Visualize the results on the frame
    annotated_frame = results[0].plot()

    # Write the frame to the output video
    out.write(annotated_frame)

    # Uncomment to display the processing (will be slower)
    # cv2.imshow("Weapon Detection", annotated_frame)
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Video processing complete! Output saved to {output_video_path}")
print(f"Total processing time: {time.time() - start_time:.2f} seconds")

# **Realtime Use**

In [8]:
from ultralytics import YOLO
import cv2
import yaml
import numpy as np
import time
from IPython.display import display, Javascript, HTML
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import PIL
import io
import html
import threading
import IPython

# Global flag to control the detection loop
STOP_DETECTION = False

# Load your trained model - update path to your best weights
model_path = "/content/drive/MyDrive/Pengolahan Citra Digital/test_detect/best (1).pt"
model = YOLO(model_path)

# Load class names
with open("/content/drive/MyDrive/Pengolahan Citra Digital/combined_gunsnknifes/data.yaml", 'r') as f:
    data = yaml.safe_load(f)
    class_names = data['names']
    print("Classes:", class_names)

# JavaScript to access webcam
js_code = """
async function captureFrame() {
    // Check if webcam access is already initialized
    if (!window.hasOwnProperty('webcamCapture')) {
        // Create video element for webcam
        const video = document.createElement('video');
        video.style.display = 'none';
        document.body.appendChild(video);

        // Create canvas for capturing frames
        const canvas = document.createElement('canvas');
        canvas.style.display = 'none';
        document.body.appendChild(canvas);

        // Store elements globally
        window.webcamCapture = {
            video: video,
            canvas: canvas,
            initialized: false
        };
    }

    const {video, canvas, initialized} = window.webcamCapture;

    // Initialize webcam if not already done
    if (!initialized) {
        try {
            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            video.srcObject = stream;
            video.play();
            // Wait for video to be ready
            await new Promise(resolve => video.onloadedmetadata = resolve);
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            window.webcamCapture.initialized = true;
        } catch (error) {
            console.error('Error accessing webcam:', error);
            return null;
        }
    }

    // Capture current frame
    const ctx = canvas.getContext('2d');
    ctx.drawImage(video, 0, 0, canvas.width, canvas.height);
    const dataUrl = canvas.toDataURL('image/jpeg');

    return {
        dataUrl: dataUrl,
        width: canvas.width,
        height: canvas.height
    };
}
captureFrame();
"""

# Function to stop the webcam stream
js_stop_webcam = """
function stopWebcam() {
    if (window.hasOwnProperty('webcamCapture') && window.webcamCapture.video.srcObject) {
        const tracks = window.webcamCapture.video.srcObject.getTracks();
        tracks.forEach(track => track.stop());
        return true;
    }
    return false;
}
stopWebcam();
"""

# Function to convert base64 to cv2 image
def js_to_image(js_reply):
    if not js_reply or not js_reply.get('dataUrl'):
        return None

    # Extract the base64 data
    b64_data = js_reply['dataUrl'].split(',')[1]
    bytes_data = b64decode(b64_data)

    # Convert to cv2 image (numpy array)
    img = PIL.Image.open(io.BytesIO(bytes_data))
    return cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

# Function to convert cv2 image to base64 for display
def image_to_b64(img):
    _, buffer = cv2.imencode('.jpg', img)
    jpg_as_text = b64encode(buffer).decode('utf-8')
    return f'data:image/jpeg;base64,{jpg_as_text}'

# Function to perform real-time detection
def real_time_detection(conf_threshold=0.30, iou_threshold=0.45):
    global STOP_DETECTION
    STOP_DETECTION = False
    frame_count = 0

    # Create display elements
    display_html = HTML("""
    <div style="display: flex; justify-content: center;">
        <img id="webcam-output" style="width: 640px;"/>
    </div>
    <div id="status" style="text-align: center; font-weight: bold; margin-top: 10px;">Initializing webcam...</div>
    <div style="text-align: center; margin-top: 10px;">
        To stop detection, run the following in a new cell: <code>stop_detection()</code>
    </div>
    """)
    display(display_html)

    print("Webcam detection started. Run stop_detection() in a new cell to stop.")

    while not STOP_DETECTION:
        try:
            # Capture frame from webcam
            js_reply = eval_js(js_code)
            if not js_reply:
                print("Failed to access webcam. Please check your browser permissions.")
                break

            # Convert to cv2 image
            frame = js_to_image(js_reply)
            if frame is None:
                continue

            # Process frame with YOLO model
            start_time = time.time()
            results = model(frame, conf=conf_threshold, iou=iou_threshold)
            inference_time = time.time() - start_time

            # Draw results on frame
            annotated_frame = results[0].plot()

            # Add FPS information
            fps = 1.0 / inference_time
            frame_count += 1
            cv2.putText(annotated_frame, f"FPS: {fps:.1f}", (20, 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Convert to base64 for display
            img_data = image_to_b64(annotated_frame)

            # Update display
            eval_js(f"""
            document.getElementById('webcam-output').src = "{img_data}";
            document.getElementById('status').innerText = "Frame {frame_count} - FPS: {fps:.1f}";
            """)

            # Short delay to allow for display update and reduce CPU usage
            time.sleep(0.01)

        except Exception as e:
            print(f"Error: {e}")
            break

    # Clean up webcam when done
    try:
        eval_js(js_stop_webcam)
    except:
        pass

    print("Real-time detection stopped.")

# Function to stop detection from another cell
def stop_detection():
    global STOP_DETECTION
    STOP_DETECTION = True
    print("Stopping detection. Please wait...")

# Register the stop function so it can be called from other cells
IPython.get_ipython().user_ns["stop_detection"] = stop_detection

# Start the real-time detection
real_time_detection()


Classes: ['pistol', 'knife']


Webcam detection started. Run stop_detection() in a new cell to stop.

0: 480x640 (no detections), 8.9ms
Speed: 1.7ms preprocess, 8.9ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)


KeyboardInterrupt: 

In [ ]:
stop_detection()

Stopping detection. Please wait...


In [6]:
from ultralytics import YOLO
import cv2
import yaml
import numpy as np
import time
from IPython.display import display, Javascript, HTML
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import PIL
import io
import threading
import IPython

# Global flag to control the detection loop
STOP_DETECTION = False

# Load your trained model - update path to your best weights
model_path = "/content/drive/MyDrive/Pengolahan Citra Digital/test_detect/best (1).pt"
model = YOLO(model_path)

# Load class names
with open("/content/drive/MyDrive/Pengolahan Citra Digital/combined_gunsnknifes/data.yaml", 'r') as f:
    data = yaml.safe_load(f)
    class_names = data['names']
    print("Classes:", class_names)

# JavaScript to access webcam
js_code = """
async function captureFrame() {
    // Check if webcam access is already initialized
    if (!window.hasOwnProperty('webcamCapture')) {
        // Create video element for webcam
        const video = document.createElement('video');
        video.style.display = 'none';
        document.body.appendChild(video);

        // Create canvas for capturing frames
        const canvas = document.createElement('canvas');
        canvas.style.display = 'none';
        document.body.appendChild(canvas);

        // Store elements globally
        window.webcamCapture = {
            video: video,
            canvas: canvas,
            initialized: false
        };
    }

    const {video, canvas, initialized} = window.webcamCapture;

    // Initialize webcam if not already done
    if (!initialized) {
        try {
            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            video.srcObject = stream;
            video.play();
            // Wait for video to be ready
            await new Promise(resolve => video.onloadedmetadata = resolve);
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            window.webcamCapture.initialized = true;
        } catch (error) {
            console.error('Error accessing webcam:', error);
            return null;
        }
    }

    // Capture current frame
    const ctx = canvas.getContext('2d');
    ctx.drawImage(video, 0, 0, canvas.width, canvas.height);
    const dataUrl = canvas.toDataURL('image/jpeg');

    return {
        dataUrl: dataUrl,
        width: canvas.width,
        height: canvas.height
    };
}
captureFrame();
"""

# Function to stop the webcam stream
js_stop_webcam = """
function stopWebcam() {
    if (window.hasOwnProperty('webcamCapture') && window.webcamCapture.video.srcObject) {
        const tracks = window.webcamCapture.video.srcObject.getTracks();
        tracks.forEach(track => track.stop());
        return true;
    }
    return false;
}
stopWebcam();
"""

# Function to convert base64 to cv2 image
def js_to_image(js_reply):
    if not js_reply or not js_reply.get('dataUrl'):
        return None

    # Extract the base64 data
    b64_data = js_reply['dataUrl'].split(',')[1]
    bytes_data = b64decode(b64_data)

    # Convert to cv2 image (numpy array)
    img = PIL.Image.open(io.BytesIO(bytes_data))
    return cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

# Function to convert cv2 image to base64 for display
def image_to_b64(img):
    _, buffer = cv2.imencode('.jpg', img)
    jpg_as_text = b64encode(buffer).decode('utf-8')
    return f'data:image/jpeg;base64,{jpg_as_text}'

# Simple tracker class for object tracking
class SimpleTracker:
    def __init__(self, max_disappeared=30, max_distance=50):
        self.nextObjectID = 0
        self.objects = {}  # Dictionary of centroids with format {ID: (x, y)}
        self.disappeared = {}  # Dictionary to count frames an object has been missing
        self.max_disappeared = max_disappeared
        self.max_distance = max_distance
        self.object_data = {}  # Store additional data about objects

    def register(self, centroid, bbox, class_id):
        # Register a new object
        self.objects[self.nextObjectID] = centroid
        self.disappeared[self.nextObjectID] = 0
        self.object_data[self.nextObjectID] = {
            'bbox': bbox,
            'class_id': class_id,
            'history': [centroid]
        }
        self.nextObjectID += 1

    def deregister(self, objectID):
        # Deregister an object
        del self.objects[objectID]
        del self.disappeared[objectID]
        del self.object_data[objectID]

    def update(self, detections):
        # detections: list of [bbox, class_id, confidence]
        # Returns dictionary of tracked objects

        # If no detections, mark all existing objects as disappeared
        if len(detections) == 0:
            for objectID in list(self.disappeared.keys()):
                self.disappeared[objectID] += 1

                # Deregister if object has been missing for too long
                if self.disappeared[objectID] > self.max_disappeared:
                    self.deregister(objectID)

            return self.object_data

        # If we're currently not tracking any objects, register all detections
        if len(self.objects) == 0:
            for detection in detections:
                bbox, class_id, confidence = detection
                x1, y1, x2, y2 = bbox
                centroid = ((x1 + x2) // 2, (y1 + y2) // 2)
                self.register(centroid, bbox, class_id)
        else:
            # Get centroids of current detections
            new_centroids = []
            for detection in detections:
                bbox, class_id, confidence = detection
                x1, y1, x2, y2 = bbox
                centroid = ((x1 + x2) // 2, (y1 + y2) // 2)
                new_centroids.append((centroid, bbox, class_id, confidence))

            # Match existing objects to new detections
            object_ids = list(self.objects.keys())
            object_centroids = list(self.objects.values())

            # Calculate distances between each pair of existing and new centroids
            distances = {}
            for i, objectID in enumerate(object_ids):
                distances[objectID] = {}
                for j, (centroid, bbox, class_id, confidence) in enumerate(new_centroids):
                    # Use Euclidean distance
                    d = np.sqrt((object_centroids[i][0] - centroid[0])**2 +
                               (object_centroids[i][1] - centroid[1])**2)
                    distances[objectID][j] = d

            # Find the smallest distance for each object and update if it's below threshold
            used_detections = set()
            for objectID in object_ids:
                if not distances[objectID]:  # Skip if no distances for this object
                    continue

                # Find closest detection
                min_dist_idx = min(distances[objectID], key=distances[objectID].get)
                min_dist = distances[objectID][min_dist_idx]

                # Only update if distance is below threshold
                if min_dist <= self.max_distance and min_dist_idx not in used_detections:
                    centroid, bbox, class_id, confidence = new_centroids[min_dist_idx]
                    self.objects[objectID] = centroid
                    self.disappeared[objectID] = 0
                    self.object_data[objectID]['bbox'] = bbox
                    self.object_data[objectID]['class_id'] = class_id
                    self.object_data[objectID]['confidence'] = confidence
                    self.object_data[objectID]['history'].append(centroid)
                    # Keep history at a reasonable length
                    if len(self.object_data[objectID]['history']) > 30:
                        self.object_data[objectID]['history'] = self.object_data[objectID]['history'][-30:]
                    used_detections.add(min_dist_idx)
                else:
                    # Mark as disappeared if no matching detection found
                    self.disappeared[objectID] += 1

            # Check for disappeared objects
            for objectID in list(self.disappeared.keys()):
                if self.disappeared[objectID] > self.max_disappeared:
                    self.deregister(objectID)

            # Register new detections that weren't matched
            for i, (centroid, bbox, class_id, confidence) in enumerate(new_centroids):
                if i not in used_detections:
                    self.register(centroid, bbox, class_id)

        return self.object_data

# Initialize tracker
tracker = SimpleTracker(max_disappeared=10, max_distance=100)

# Function to perform real-time detection with tracking
def real_time_detection_with_tracking(conf_threshold=0.30, iou_threshold=0.45):
    global STOP_DETECTION
    STOP_DETECTION = False
    frame_count = 0

    # Track colors for visualization
    def get_color(track_id):
        # Generate a color based on track ID (consistent across frames)
        np.random.seed(track_id * 123)
        return (int(np.random.randint(0, 255)),
                int(np.random.randint(0, 255)),
                int(np.random.randint(0, 255)))

    # Create display elements
    display_html = HTML("""
    <div style="display: flex; justify-content: center;">
        <img id="webcam-output" style="width: 640px;"/>
    </div>
    <div id="status" style="text-align: center; font-weight: bold; margin-top: 10px;">Initializing webcam...</div>
    <div style="text-align: center; margin-top: 10px;">
        To stop detection, run the following in a new cell: <code>stop_detection()</code>
    </div>
    """)
    display(display_html)

    print("Webcam detection with tracking started. Run stop_detection() in a new cell to stop.")

    while not STOP_DETECTION:
        try:
            # Capture frame from webcam
            js_reply = eval_js(js_code)
            if not js_reply:
                print("Failed to access webcam. Please check your browser permissions.")
                break

            # Convert to cv2 image
            frame = js_to_image(js_reply)
            if frame is None:
                continue

            # Process frame with YOLO model
            start_time = time.time()
            results = model(frame, conf=conf_threshold, iou=iou_threshold)
            inference_time = time.time() - start_time

            # Prepare detections for tracker
            detections = []
            for box in results[0].boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
                confidence = float(box.conf[0].cpu().numpy())
                class_id = int(box.cls[0].cpu().numpy())
                detections.append([(x1, y1, x2, y2), class_id, confidence])

            # Update tracker
            tracked_objects = tracker.update(detections)

            # Create a copy of the frame for visualization
            tracked_frame = frame.copy()

            # Visualize the tracked objects
            for track_id, data in tracked_objects.items():
                bbox = data['bbox']
                class_id = data['class_id']
                history = data['history']

                # Get color based on track ID
                color = get_color(track_id)

                # Draw bounding box
                cv2.rectangle(tracked_frame, (bbox[0], bbox[1]), (bbox[2], bbox[3]), color, 2)

                # Add label with track ID and class
                class_name = class_names[class_id]
                label = f"{class_name}-{track_id}"
                cv2.putText(tracked_frame, label, (bbox[0], bbox[1] - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

                # Draw tracking lines (history)
                if len(history) > 1:
                    for i in range(1, len(history)):
                        pt1 = history[i-1]
                        pt2 = history[i]
                        cv2.line(tracked_frame, pt1, pt2, color, 2)

            # Add FPS information
            fps = 1.0 / inference_time
            frame_count += 1
            cv2.putText(tracked_frame, f"FPS: {fps:.1f}", (20, 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Convert to base64 for display
            img_data = image_to_b64(tracked_frame)

            # Update display
            eval_js(f"""
            document.getElementById('webcam-output').src = "{img_data}";
            document.getElementById('status').innerText = "Frame {frame_count} - FPS: {fps:.1f} - Tracking {len(tracked_objects)} objects";
            """)

            # Short delay to allow for display update
            time.sleep(0.01)

        except Exception as e:
            print(f"Error: {e}")
            import traceback
            traceback.print_exc()
            break

    # Clean up webcam when done
    try:
        eval_js(js_stop_webcam)
    except:
        pass

    print("Real-time detection with tracking stopped.")

# Function to stop detection from another cell
def stop_detection():
    global STOP_DETECTION
    STOP_DETECTION = True
    print("Stopping detection. Please wait...")

# Register the stop function so it can be called from other cells
IPython.get_ipython().user_ns["stop_detection"] = stop_detection

# Start the real-time detection with tracking
real_time_detection_with_tracking()

Classes: ['pistol', 'knife']


Webcam detection with tracking started. Run stop_detection() in a new cell to stop.

0: 480x640 (no detections), 40.0ms
Speed: 9.9ms preprocess, 40.0ms inference, 104.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 9.4ms
Speed: 4.5ms preprocess, 9.4ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 14.8ms
Speed: 2.0ms preprocess, 14.8ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 9.0ms
Speed: 2.1ms preprocess, 9.0ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 7.4ms
Speed: 2.0ms preprocess, 7.4ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 9.1ms
Speed: 1.9ms preprocess, 9.1ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 9.8ms
Speed: 1.9ms preprocess, 9.8ms inference, 0.9ms postprocess per image at shape (1, 3, 480

KeyboardInterrupt: 

In [ ]:
stop_detection

<function __main__.stop_detection()>